In [12]:
import math
from scipy.stats import norm

def calculate_mde_one_sided(weekly_traffic,
                            weekly_conversions,
                            confidence_level,
                            power,
                            num_variants,
                            weeks=[1,2,3,4,5,6]):
    
    # Convert confidence level and power to Z-scores for one-sided test
    z_alpha = norm.ppf(confidence_level)  # One-sided, so no division by 2
    z_beta = norm.ppf(power) 

    baseline_cr = weekly_conversions / weekly_traffic  # Baseline conversion rate
    
    mde_list = []
    
    for w in weeks:
        # Number of users per variant after w weeks
        total_users_per_variant = (weekly_traffic / num_variants) * w
        
        # Calculate MDE for w weeks (absolute)
        mde = z_alpha * math.sqrt((2 * baseline_cr * (1 - baseline_cr)) / total_users_per_variant) + \
              z_beta * math.sqrt((2 * baseline_cr * (1 - baseline_cr)) / total_users_per_variant)
        
        # Calculate relative MDE as a percentage of the baseline conversion rate
        relative_mde = (mde / baseline_cr) * 100
        
        # Append week, absolute MDE, relative MDE, and visitors per variant
        mde_list.append((w, mde, relative_mde, total_users_per_variant))
    
    return mde_list

# Example usage:
weekly_traffic = 10000  # Weekly traffic (sessions or users)
weekly_conversions =300  # Weekly conversions
num_variants = 2  # Number of variants (including control)
baseline_cr = 0.05  # 60% baseline conversion rate
confidence_level = 0.95  # One-sided confidence level (95%)
power = 0.80  # Statistical power (80%)

mde_results = calculate_mde_one_sided(weekly_traffic, weekly_conversions, confidence_level, power, num_variants)

# Print results
print(f"{'Week':<6}{'MDE (%)':<10}{'Relative MDE (%)':<20}{'Visitors per Variant':<20}")
for week, mde, relative_mde, visitors_per_variant in mde_results:
    print(f"{week:<6}{mde:.2%}     {relative_mde:.2f}%              {int(visitors_per_variant):<20}")

Week  MDE (%)   Relative MDE (%)    Visitors per Variant
1     0.85%     28.28%              5000                
2     0.60%     20.00%              10000               
3     0.49%     16.33%              15000               
4     0.42%     14.14%              20000               
5     0.38%     12.65%              25000               
6     0.35%     11.54%              30000               
